In [34]:
import pandas as pd
import pickle
from sklearn import linear_model
import numpy as np
import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot


#Output of the Prediction for matchday (9 games)
# Form: Home_team, Away_team, Prediction
# Prediction over 0.5 is a Victory for Home Team
# Prediction under 0.5 is a Loss for Home Team
# Else Draw
def ausgabe(mannschhaft1,mannschaft2,predition):
    gesamt=[]
    for i in range(0,9):
        game= [mannschhaft1[i],mannschaft2[i],predition[i]]
        gesamt.append(game)
    return gesamt

#Modification: Home Team gets an 1, Away Team a -1
#Idea: Home Team has an advantage with fans
def tabellespiele(dfmod):
    df_visitor = pd.get_dummies(dfmod['Mannschaft2'], dtype=np.int64)
    df_visitor= df_visitor*-1
    df_home = pd.get_dummies(dfmod['Mannschaft1'], dtype=np.int64)
    
    #print(dfmod,len(dfmod))
    df_model=pd.concat([df_home,df_visitor],axis=1)

    
    return df_model

#Modification: Home Teams gets a 1, Away Team -1
#Iterate through each line
#Fix Nan Values
def tabelle(dfmod,anfrage,anfang,ende):
    #Wenn nur ein Spieltag zu betrachten ist, Methode tabellespiele aurufen
    #um NAN Fehler zu vermeiden (nur 9 Mannschaften in M1 und M2)
    if (ende-anfang)==9:
        df_model= tabellespiele(dfmod)
        for i in anfrage:
            df_model[i] = dfmod[i]
        return df_model

    df_visitor = pd.get_dummies(dfmod['Mannschaft2'], dtype=np.int64)
    df_home = pd.get_dummies(dfmod['Mannschaft1'], dtype=np.int64)
    #print(df_visitor)
    #Fehlerkorrektur in der Tabelle (NaN Fehler)
    nan_home=[]
    for (columnName, columnData) in df_home.iteritems():
        team= columnName
        if np.sum(columnData.values)>1:
            teamwerte= [team,columnData.values]
            nan_home.append(teamwerte)

    nan_gast=[]
    for (columnName, columnData) in df_visitor.iteritems():
        team= columnName
        if np.sum(columnData.values)>1:
            columnData = columnData*-1
            teamwerte= [team,columnData.values]
            nan_gast.append(teamwerte)
            #nan_gast.append(columnData.values)        

    #Transformation: 1:Heimmannschaft, -1: Gastmannschaft und hinzunahme von Differenz
    df_model = df_home.sub(df_visitor)
    for i in anfrage:
        df_model[i] = dfmod[i]
    #Wenn eine Mannschaft nur Heimspiele hat muessen die Werte umgeschrieben,
    #sonst nan in der Spalte und man kann nicht weiter arbeiten
    
    count=0
    for (columnName, columnData) in df_model.iteritems():

        if np.isnan(np.sum(columnData.values)):
            for i in nan_home:
                print(i[0])
                if columnName==i[0]:
                    df_model[columnName]= i[1]
            for i in nan_gast:
                print(i[0])
                if columnName==i[0]:
                    df_model[columnName]=i[1]
    #print("---")
    #print(df_model)
    return df_model

#Collect the data of the pkl. to df
#Each Feature is a List
#Take only the Data of the Features (arg: anfrage) for an area of matchdays (arg: anfang, ende)
def einlesen(spieldaten,anfang,ende,anfrage):    
    mannschaft1=[]
    mannschaft2=[]
    tore= []
    schuesse=[]
    anfrageliste_e=[]
    anfrageliste_f=[]
    anfrageliste_b=[]
    anfrageliste_p=[]
    anfrageliste_l=[]
    anfrageliste_z=[]
    anfrageliste_s=[]
    anfrageliste_paesse=[]
    anfrageliste_ep=[]
    anfrageliste_fp=[]
    anfrageliste_gfoul=[]
    anfrageliste_a=[]
    anfrageliste_ecken=[]
    anfrageliste_komplett=[]
    ergebnisse=[]

    count=0
    dfmod= ""
    with (open(spieldaten, "rb")) as openfile:
        while True:
            try:
                if count< ende:
                    spieltag= pickle.load(openfile)
                    #print(spieltag)
                    #print(spieltag[1], spieltag[2], spieltag[4][0:2], spieltag[5][0:2])
                    if spieltag[1]=="Mannschaft1":
                        continue
                    if count >= anfang:
                        mannschaft1.append(spieltag[1])
                        mannschaft2.append(spieltag[2])
                        for i in anfrage:
                            #print(i)
                            if i=="effizienz":
                                eff_heim= int(spieltag[4][0])/int(spieltag[4][1])
                                eff_gast= int(spieltag[5][0])/int(spieltag[5][1])
                                effi_gesamt= eff_heim-eff_gast
                                #print(effi_gesamt)
                                anfrageliste_e.append(effi_gesamt)
                            elif i=="ballbesitz":
                                ballbesitz_heim= int(spieltag[4][7][:-1])
                                ballbesitz_gast= int(spieltag[5][7][:-1])
                                ballbesitz_gesamt= ballbesitz_heim/ballbesitz_gast
                                anfrageliste_b.append(ballbesitz_gesamt)
                            elif i=="schuesse":
                                schuesse_heim= int(spieltag[4][1])
                                schuesse_gast= int(spieltag[5][1])
                                schuesse_gesamt= schuesse_heim/schuesse_gast
                                anfrageliste_s.append(schuesse_gesamt)
                            elif i=="paesse":
                                paesse_heim= int(spieltag[4][3])
                                paesse_gast= int(spieltag[5][3])
                                paesse_gesamt= paesse_heim/paesse_gast
                                anfrageliste_paesse.append(paesse_gesamt)
                            elif i=="epaesse":
                                epaesse_heim= int(spieltag[4][4])
                                epaesse_gast= int(spieltag[5][4])
                                epaesse_gesamt= epaesse_heim/epaesse_gast
                                anfrageliste_ep.append(epaesse_gesamt)
                            elif i=="fpaesse":
                                fpaesse_heim= int(spieltag[4][5])
                                fpaesse_gast= int(spieltag[5][5])
                                fpaesse_gesamt= fpaesse_heim/fpaesse_gast
                                anfrageliste_fp.append(fpaesse_gesamt)
                            elif i=="foul":
                                foul_heim= int(spieltag[4][9])
                                foul_gast= int(spieltag[5][9])
                                foul_gesamt= foul_heim/foul_gast
                                anfrageliste_f.append(foul_gesamt)
                            elif i=="gfoul":
                                gfoul_heim= int(spieltag[4][10])
                                gfoul_gast= int(spieltag[5][10])
                                gfoul_gesamt= gfoul_heim/gfoul_gast
                                anfrageliste_gfoul.append(gfoul_gesamt)
                            elif i=="abseits":
                                a_heim= int(spieltag[4][11])
                                a_gast= int(spieltag[5][11])
                                if a_gast==0:
                                    a_gast=1
                                if a_heim==0:
                                    a_heim=1
                                
                                a_gesamt= a_heim/a_gast
                                anfrageliste_a.append(a_gesamt)
                            elif i=="ecken":
                                ecken_heim= int(spieltag[4][12])
                                ecken_gast= int(spieltag[5][12])
                                if ecken_heim==0:
                                    ecken_heim=1
                                if ecken_gast==0:
                                    ecken_gast=1
                                ecken_gesamt= ecken_heim/ecken_gast
                                anfrageliste_ecken.append(ecken_gesamt)
                            elif i=="laufen":
                                km_heim= float((spieltag[4][2][:-2]).replace(",","."))
                                km_gast= float((spieltag[5][2][:-2]).replace(",","."))
                                lauf_gesamt= km_heim/km_gast
                                anfrageliste_l.append(lauf_gesamt)
                            elif i=="passen":
                                passen_heim= int(spieltag[4][6][:-1])
                                passen_gast= int(spieltag[5][6][:-1])
                                passen_gesamt= passen_heim/passen_gast
                                anfrageliste_p.append(passen_gesamt)
                                #print(passen_gesamt)
                            elif i=="zweikampf":
                                zweikampf_heim= int(spieltag[4][8][:-1])
                                zweikampf_gast= int(spieltag[5][8][:-1])
                                zweikampf_gesamt= zweikampf_heim/zweikampf_gast
                                anfrageliste_z.append(zweikampf_gesamt)
                            
            
                        ergebnis= (spieltag[4][0:1][0])+":"+(spieltag[5][0:1][0])
                        ergebnisse.append(ergebnis)
                        data= {"Mannschaft1":mannschaft1,"Mannschaft2":mannschaft2}
                        #print(anfrageliste_komplett)
                        for i in anfrage:
                            if i=="effizienz":
                                data.update({i:anfrageliste_e})
                            if i=="foul":
                                data.update({i:anfrageliste_f})
                            if i=="laufen":
                                data.update({i:anfrageliste_l})
                            if i=="passen":
                                data.update({i:anfrageliste_p})
                            if i=="zweikampf":
                                data.update({i:anfrageliste_z})
                            if i=="schuesse":
                                data.update({i:anfrageliste_s})
                            if i=="paesse":
                                data.update({i:anfrageliste_paesse})
                            if i=="epaesse":
                                data.update({i:anfrageliste_ep})
                            if i=="fpaesse":
                                data.update({i:anfrageliste_fp})
                            if i=="ballbesitz":
                                data.update({i:anfrageliste_b})
                            if i=="gfoul":
                                data.update({i:anfrageliste_gfoul})
                            if i=="abseits":
                                data.update({i:anfrageliste_a})
                            if i=="ecken":
                                data.update({i:anfrageliste_ecken})
                            
                        #print(data)

                        dfmod= pd.DataFrame(data)
                        
                    count=count+1
                else:
                    break
            except EOFError:
                break
    return dfmod, mannschaft1,mannschaft2,ergebnisse


def ytransform(y,art):
    if art=="TSNE":
        return TSNE(n_components=1).fit_transform(y)
    elif art=="PCA":
        return PCA(n_components=1).fit_transform(y)
    return "Fehler"

#Collect all Data whitch are quiered.
#Predict with Linear Regression
def Statistik(spieldaten, anfang,ende,anfrage):
    
    anfang=anfang*9
    ende=ende*9 
    dfmod,m1,m2,e1= einlesen(spieldaten,anfang,ende,anfrage)
    
    #print(dfmod)
    df_visitor = pd.get_dummies(dfmod['Mannschaft2'], dtype=np.int64)
    df_home = pd.get_dummies(dfmod['Mannschaft1'], dtype=np.int64)
    df_model= tabelle(dfmod,anfrage,anfang,ende)
    '''Verwendung von Ridge Regression
        wird verwendet wenn es viele Features besitzt oder wenige Trainingsdaten besitzt
        Vermeidung von Un- oder Ueberanpassung'''

    lr = sklearn.linear_model.Ridge(alpha=0.001)
    rt= RandomForestRegressor()
    X = df_model.drop(anfrage,axis=1)
    #print(X)
    #y-Achse bilden
    if len(anfrage) ==1:
        feature= anfrage[0]
        y= df_model[feature]
        #print(y)
    else:
        y = np.vstack((df_model[anfrage[0]],df_model[anfrage[1]]))
        for i in range(2,len(anfrage)):
            if len(anfrage) <=2:
                break
            else:
                y = np.vstack((y,df_model[anfrage[i]]))
    #print(y)        
    y = y.transpose()
    #print(y)
    # Das gefittete PCA braucht man um andere (Test, oder neue Daten) zu transformen
    # andere wären t-SNE, UMAP
    if len(anfrage)==1:
        y= y.values.reshape(-1,1)
    #print(y)
    y = ytransform(y,"PCA")
    #print(y)
    y = y.reshape(y.shape[0])
    lr.fit(X, y)
    #rt.fit(X,y)

    
    #df_ratings = pd.DataFrame(data={'team': X.columns, 'rating': lr.coef_})
    #coef= df_ratings.values

    #Spiele fuer das naechste Spiel holen und Tabelle erstellen
    dfmod2,m1,m2,ergebnisse= einlesen(spieldaten,ende,ende+9,anfrage)
    df_model2= tabellespiele(dfmod2)
    df_model2 = pd.DataFrame(data=df_model2)
    prediction= lr.predict(df_model2)
    #prediction=rt.predict(df_model2)
    gesamt = ausgabe(m1,m2,prediction)
    return gesamt

#Used Random Forest for Feature Selection
#Which Feature is the best feature to predict football results
def RandomForest(spieldaten, anfang,ende,anfrage):
    
    anfang=anfang*9
    ende=ende*9 
    dfmod,m1,m2,e1= einlesen(spieldaten,anfang,ende,anfrage)
    X=dfmod.drop(["Mannschaft1","Mannschaft2"],axis=1)
    y=[]
    for i in e1:
        numbers= str(i).split(":")
        diff = int(numbers[0])-int(numbers[1])
        if diff>0:
            y.append(1)
        elif diff==0:
            y.append(0)
        elif diff<0:
            y.append(-1)
    y= np.array(y)
    model = RandomForestClassifier(n_estimators=5000, max_features=len(anfrage))
    # fit the model on the whole dataset
    model.fit(X, y)
    #print(model.feature_importances_)
    return model.feature_importances_

#Choice Values for Prediction (Features, area of match days, season of football)
#Print Prediction for football game
def main():
    tanfrage=["schuesse","ballbesitz","gfoul","paesse"]
    #tanfrage=["schuesse","ecken","ballbesitz","gfoul","paesse","epaesse","abseits","fpaesse","foul","passen","zweikampf","laufen"]

    tanfang=[1]
    tende=[33]

    for i in range(0,len(tanfang)):
        anfang_neu=tanfang[i]
        ende_neu=tende[i]
        liste= RandomForest("Statistik2019.pkl", anfang_neu,ende_neu,tanfrage)
        features={}
        for i in range(0,len(liste)):
            features[tanfrage[i]]=liste[i]
        sortierte_features=(sorted(features.items(), key=lambda x: x[1],reverse=True))
        tanfrage_neu=[]
        for i in sortierte_features:
            tanfrage_neu.append(i[0])
            #print(i[0],i[1])
        #tanfrage_neu.pop()
        #tanfrage_neu.pop()
        liste= Statistik("Statistik2019.pkl", anfang_neu,ende_neu,tanfrage_neu)
        print("Bertrachtung der Spiele von "+str(anfang_neu)+" bis "+str(ende_neu)+"\n")
        for i in liste:
            print(i)
        print("\n")
